## Practices 1

In [1]:
import findspark
findspark.init()
findspark.find()
from pyspark.sql import SparkSession
from pyspark.sql.functions import mean,col,split, col, regexp_extract, when, lit

In [2]:
#creat session
spark = SparkSession.builder.appName("SimpleApp").getOrCreate()

In [3]:
path = "data/titanic_proj_train.csv"
titanic_df = spark.read.csv(path,header = 'True',inferSchema='True')

In [4]:
display(titanic_df)

DataFrame[PassengerId: int, Survived: int, Pclass: int, Name: string, Sex: string, Age: double, SibSp: int, Parch: int, Ticket: string, Fare: double, Cabin: string, Embarked: string]

In [5]:
titanic_df.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [6]:
titanic_df.show(5)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| NULL|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| NULL|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| NULL|       S|
+-----------+--------+------+--------------------+------+----+-----+-----+------

In [7]:
titanic_df.select("Survived","Pclass","Embarked").show()

+--------+------+--------+
|Survived|Pclass|Embarked|
+--------+------+--------+
|       0|     3|       S|
|       1|     1|       C|
|       1|     3|       S|
|       1|     1|       S|
|       0|     3|       S|
|       0|     3|       Q|
|       0|     1|       S|
|       0|     3|       S|
|       1|     3|       S|
|       1|     2|       C|
|       1|     3|       S|
|       1|     1|       S|
|       0|     3|       S|
|       0|     3|       S|
|       0|     3|       S|
|       1|     2|       S|
|       0|     3|       Q|
|       1|     2|       S|
|       0|     3|       S|
|       1|     3|       C|
+--------+------+--------+
only showing top 20 rows



In [8]:
titanic_df.groupBy("Survived").count().show()

+--------+-----+
|Survived|count|
+--------+-----+
|       1|  308|
|       0|  491|
+--------+-----+



In [9]:
titanic_df.groupBy("Pclass","Survived").count().show()

+------+--------+-----+
|Pclass|Survived|count|
+------+--------+-----+
|     1|       0|   75|
|     3|       1|  108|
|     1|       1|  120|
|     2|       1|   80|
|     2|       0|   86|
|     3|       0|  330|
+------+--------+-----+



In [10]:
# task 1
#Did more male or а female survived? (show the table)
survived_data_male_female = titanic_df.groupby(['Sex', 'Survived']).count()
survived_data = survived_data_male_female[survived_data_male_female['Survived'] == 1]

survived_data.show()
# Answer => Male who survived were 97 and Female who survived were 211

+------+--------+-----+
|   Sex|Survived|count|
+------+--------+-----+
|female|       1|  211|
|  male|       1|   97|
+------+--------+-----+



In [11]:
#check Initial
titanic_df.select("Name").show()

+--------------------+
|                Name|
+--------------------+
|Braund, Mr. Owen ...|
|Cumings, Mrs. Joh...|
|Heikkinen, Miss. ...|
|Futrelle, Mrs. Ja...|
|Allen, Mr. Willia...|
|    Moran, Mr. James|
|McCarthy, Mr. Tim...|
|Palsson, Master. ...|
|Johnson, Mrs. Osc...|
|Nasser, Mrs. Nich...|
|Sandstrom, Miss. ...|
|Bonnell, Miss. El...|
|Saundercock, Mr. ...|
|Andersson, Mr. An...|
|Vestrom, Miss. Hu...|
|Hewlett, Mrs. (Ma...|
|Rice, Master. Eugene|
|Williams, Mr. Cha...|
|Vander Planke, Mr...|
|Masselmani, Mrs. ...|
+--------------------+
only showing top 20 rows



In [12]:
titanic_df = titanic_df.withColumn("Initial",regexp_extract(col("Name"),"([A-Za-z]+)\.",1))
titanic_df.select("Initial").show()

+-------+
|Initial|
+-------+
|     Mr|
|    Mrs|
|   Miss|
|    Mrs|
|     Mr|
|     Mr|
|     Mr|
| Master|
|    Mrs|
|    Mrs|
|   Miss|
|   Miss|
|     Mr|
|     Mr|
|   Miss|
|    Mrs|
| Master|
|     Mr|
|    Mrs|
|    Mrs|
+-------+
only showing top 20 rows



In [13]:
titanic_df = titanic_df.replace(['Mlle','Mme', 'Ms', 'Dr','Major','Lady','Countess','Jonkheer','Col','Rev','Capt','Sir','Don'],
               ['Miss','Miss','Miss','Mr','Mr',  'Mrs',  'Mrs',  'Other',  'Other','Other','Mr','Mr','Mr'])

In [14]:
titanic_df.select("Initial").distinct().show()

+-------+
|Initial|
+-------+
|   Miss|
|  Other|
| Master|
|     Mr|
|    Mrs|
+-------+



In [15]:
#Calculate  avereg age 
titanic_df.groupby('Initial').avg('Age').collect()

[Row(Initial='Miss', avg(Age)=21.97841726618705),
 Row(Initial='Other', avg(Age)=53.333333333333336),
 Row(Initial='Master', avg(Age)=4.693571428571429),
 Row(Initial='Mr', avg(Age)=32.88274932614555),
 Row(Initial='Mrs', avg(Age)=35.78260869565217)]

In [18]:
#Task 2. What is the median and standard deviation of payments (`Fare`)?
from pyspark.sql.functions import mean, stddev , median
# Fare
fare_stddev = titanic_df.select(stddev("Fare")).collect()[0][0]

fare_median = titanic_df.select(median("Fare")).collect()[0][0]

fare_mean = titanic_df.select(mean("Fare")).collect()[0][0]


print(f"Standard deviation is $ {fare_stddev:.2f}")

print(f"Median is $ {fare_median:.2f}")

print(f"Mean is $ {fare_mean:.2f}")


Standard deviation is $ 51.56
Median is $ 14.50
Mean is $ 33.05


In [20]:
#Task 3.
#Find the most popular the male name.
from collections import Counter
# Select Name
df =  titanic_df.select("Name")[titanic_df['Sex'] == 'male'].collect()
names = []
for r in df:
    name = r["Name"].split(",")
    
    surname = name[0]
    fullname = name[1].split(".")[1].split(" ")
    f_name = fullname[1]

    
    if len(fullname) == 3:
        l_name = fullname[2]
    
    names.append(f_name)
    names.append(l_name)
    names.append(surname)    

name_counts = Counter(names)
mode_name = name_counts.most_common(1)[0][0]  # Extracts the most common name
mode_name

print(f"The most popular the male name is {mode_name}")




The most popular the male name is William


In [21]:
#Task 4.Is it true that people under the age of 30 survived more often than people over the age of 60? What are the proportions of survivors in both groups?

# Calculate:

under_30 = titanic_df[titanic_df['Age'] < 30]

over_30 = titanic_df[titanic_df['Age'] > 60]

# under 30
under_mean = under_30.select(mean("Survived")).collect()[0][0]
# Over 30
over_mean =  over_30.select(mean("Survived")).collect()[0][0]
    
print(f"People Under age of 30 did survive by {under_mean*100:.1f}%")
print(f"People Over age of 30 did survive by {over_mean*100:.1f}%")



People Under age of 30 did survive by 40.4%
People Over age of 30 did survive by 20.0%


In [22]:
#Task 5.
'''
How does the average age of men/women differ depending on the class of service? Choose the correct statements:**
- On average, men of the 1st class are over 40 years old
- On average, 1st grade women are over 40 years old
- Men of all classes are on average older than women of the same class
- On average, people in the 1st grade are older than in the 2nd, and those are older than representatives of the 3rd grade
'''
# Select Sex , Age and Pclass from the dataset
data = titanic_df.select("Sex","Age","Pclass")

# Select only first class passengers

data_class = data[data["Pclass"] == 1]

# ANS_1 => On average, men of the 1st class are over 40 years old

# Find Average of the men on the 1st class 
men_age_avg_01 = data_class[data_class["Sex"] == "male"].select(mean("Age")).collect()[0][0]

# ANS =>   On average, men of the 1st class are over 40 years old
print("QN_1:=> On average, men of the 1st class are over 40 years old")
print(f"Yes , The average age of men in the 1st class is approximately {men_age_avg_01: .2f} years.")
print("**************************")

# ANS_2
# Find Average of the women on the 1st class 
women_age_avg_01 = data_class[data_class["Sex"] == "female"].select(mean("Age")).collect()[0][0]


# ANS =>   On average, women of the 1st class are over 40 years old

print("QN_2:=> On average, 1st grade women are over 40 years old")
print(f"No , The average age of women in the 1st class is approximately {women_age_avg_01: .2f} years.")
print("**************************")


# ANS_3

#Men age average of all classes
data_male_avg = data[data["Sex"] == "male"].groupby('Pclass').mean("Age").collect()

#Women age average of all classes
data_women_avg = data[data["Sex"] == "female"].groupby('Pclass').mean("Age").collect()


# ANS => Men of all classes are on average older than women of the same class

# men_older_than_women_same_class = all(data_male_avg > data_women_avg)

# print(men_older_than_women_same_class)

age_male_avg = [row[1] for row in data_male_avg]
age_women_avg = [row[1] for row in data_women_avg]

print("QN_3:=> Men of all classes are on average older than women of the same class ")
print(f"Men   = {age_male_avg }")
print(f"Women = {age_women_avg }")
ans =  all(men_age > women_age for men_age, women_age in zip(age_male_avg, age_women_avg))
print(f"it is {ans} that all men are older than women in the same class")
print("**************************")

# ANS_4
# On average, people in the 1st grade are older than in the 2nd, 
# and those are older than representatives of the 3rd grade
classes_age_avg = data.groupby('Pclass').mean("Age").collect()

print("QN_4:=> On average, people in the 1st grade are older than in the 2nd, and those are older than representatives of the 3rd grade ")
for row in classes_age_avg:
    print(f" {row[0]} Class on average approximately {row[1]: .2f} years")
print("**************************")




QN_1:=> On average, men of the 1st class are over 40 years old
Yes , The average age of men in the 1st class is approximately  41.92 years.
**************************
QN_2:=> On average, 1st grade women are over 40 years old
No , The average age of women in the 1st class is approximately  33.72 years.
**************************
QN_3:=> Men of all classes are on average older than women of the same class 
Men   = [41.91936170212766, 26.47085201793722, 31.70689655172414]
Women = [33.71621621621622, 21.916666666666668, 28.360294117647058]
it is True that all men are older than women in the same class
**************************
QN_4:=> On average, people in the 1st grade are older than in the 2nd, and those are older than representatives of the 3rd grade 
 1 Class on average approximately  38.31 years
 3 Class on average approximately  25.16 years
 2 Class on average approximately  30.24 years
**************************
